# Processing Big Data - Data Ingestion
© Explore Data Science Academy

## Honour Code
I **NAOMI**, **NANGOEJANE**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.



## Context 

To work constructively with any dataset, one needs to create an ingestion profile to make sure that the data at the source can be readily consumed. For this section of the predict, as the Data Engineer in the team, you will be required to design and implement the ingestion process. For the purposes of the project the AWS cloud storage service, namely, the S3 bucket service will act as your data source. All the data required can be found [here](https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip).

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/DataIngestion.jpg"
     alt="Data Ingestion"
     style="float: center; padding-bottom=0.5em"
     width=40%/>
     <p><em>Figure 1. Data Ingestion</em></p>
</div>

Your manager, Gnissecorp Atadgib, knowing very well that you've recently completed your Data Engineering qualification, asks you to make use of Apache Spark for the ingestion as well as the rest of the project. His rationale being, that stock market data is generated every day and is quite time-sensitive and would require scalability when deploying to a production environment. 

## Dataset - US Nasdaq




<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/Nasdaq.png"
     alt="Nasdaq"
     style="float: center; padding-bottom=0.5em"
     width=50%/>
     <p><em>Figure 2. Nasdaq</em></p>
</div>

The data that you will be working with is a historical snapshot of market data taken from the Nasdaq electronic market. This dataset contains historical daily prices for all tickers currently trading on Nasdaq. The up-to-date list can be found on their [website](https://www.nasdaq.com/)


The provided data contains price data dating back from 02 January 1962 up until 01 April 2020. The data found in the S3 bucket has been stored in the following structure:

```
     stocks/<Year>/<Month>/<Day>/stocks.csv
```
Each CSV file for every trading day contains the following details:
- **Date** - specifies trading date
- **Open** - opening price
- **High** - maximum price during the day
- **Low** - minimum price during the day
- **Close** - close price adjusted for splits
- **Adj Close** - close price adjusted for both dividends and splits
- **Volume** - the number of shares that changed hands during a given day

## Basic initialisation
To get you started, let's import some basic Python libraries as well as Spark modules and functions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

Remember that we need a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs and the `SparkSession` to interface with Python objects.

> ℹ️ **Instructions** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [2]:
#TODO: Write your code here

sc = SparkContext().getOrCreate()
spark = SparkSession(sc)

## Investigate dataset schema
At this point, it is enough to read in a single file to ascertain the data structure. You will be required to use the information obtained from the small subset to create a data schema. This data schema will be used when reading the entire dataset using Spark.

> ℹ️ **Instructions** ℹ️
>
>Make use of Pandas to read in a single file and investigate the plausible data types to be used when creating a Spark data schema. 
>
>*You may use as many coding cells as necessary.*

In [83]:
#TODO: Write your code here
# Read a single CSV file using Pandas
df = pd.read_csv('C:/Users/Naomi/Desktop/data/stocks/1962/01/02/stocks.csv')


df.head(10)

# Investigate the data types of each column
#data_types = df.dtypes
#df = spark.read.csv('C:/Users/Naomi/Desktop/data/bitcoin_train.csv', header=True)
#df_pandas.printSchema()
#data_types

,Date,Open,High,Low,Close,Adj Close,Volume,stock
0,1962-01-02,6.532155,6.556185,6.532155,6.532155,1.536658,55900.0,AA
1,1962-01-02,6.125844,6.160982,6.125844,6.125844,1.414651,59700.0,ARNC
2,1962-01-02,0.837449,0.837449,0.823045,0.823045,0.145748,352200.0,BA
3,1962-01-02,1.604167,1.619792,1.588542,1.604167,0.136957,163200.0,CAT
4,1962-01-02,0.000000,3.296131,3.244048,3.296131,0.051993,105600.0,CVX
5,1962-01-02,0.092908,0.096026,0.092908,0.092908,0.035517,817400.0,DIS
6,1962-01-02,0.000000,30.687500,30.375000,30.375000,0.506943,1600.0,DTE
7,1962-01-02,0.000000,10.281250,10.125000,10.125000,0.227351,25600.0,ED
8,1962-01-02,0.000000,7.687500,7.541667,7.583333,0.978945,49200.0,FL
9,1962-01-02,0.751202,0.763722,0.743690,0.748698,0.001782,2156500.0,GE


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       20 non-null     object 
 1   Open       20 non-null     float64
 2   High       20 non-null     float64
 3   Low        20 non-null     float64
 4   Close      20 non-null     float64
 5   Adj Close  20 non-null     float64
 6   Volume     20 non-null     float64
 7   stock      20 non-null     object 
dtypes: float64(6), object(2)
memory usage: 1.4+ KB


## Read CSV files

When working with big data, it is often not tenable to keep processing an entire data batch when you are in the process of development - this can be quite time-consuming. If the data is uniform, it is sufficient to work with a smaller subset to create basic functionality. Your manager has identified the year **1962** to perform the initial testing for data ingestion. 

> ℹ️ **Instructions** ℹ️
>
>Read in the data for **1962** using a data schema that purely uses string data types. You will be required to convert to the appropriate data types at a later stage.
>
>*You may use as many coding cells as necessary.*

In [11]:
#TODO: Write your code here
#from pyspark.sql.types import StructType, StringType

#must read in the whole folder
parent_dir = '/content/stocks/1962')
new_df = spark.read.csv(parent_dir+ '*/*/*.csv', header=True)
new_df.show(10)


In [ ]:
#change datatype
stocks_schema = StructType([
    StructField('Date', StringType(), False),
    StructField('Open', StringType(), False),
    StructField('High', StringType()),
    StructField('Low', StringType()),
    StructField('Close', StringType()), 
    StructField('Adj Close', StringType()),
    StructField('Volume', StringType())
])

In [12]:
stocks_df = spark.read.csv('C:/Users/Naomi/Desktop/data/stocks/1962/01/02/stocks.csv', header=True, schema=stocks_schema)

In [13]:
stocks_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)



## Update column names
To make the data easier to work with, you will need to make a few changes:
1. Column headers should all be in lowercase; and
2. Whitespaces should be replaced with underscores.


> ℹ️ **Instructions** ℹ️
>
>Make sure that the column headers are all in lowercase and that any whitespaces are replaced with underscores.
>
>*You may use as many coding cells as necessary.*

In [ ]:
#Predict session use replace
for col in new_def.columns:
    nwe_df

stocks_df.printSchema()

In [14]:
#TODO: Write your code here

# Loop over each column name within our DataFrame.
for column in stocks_df.columns:
    stocks_df = stocks_df.withColumnRenamed(column, '_'.join(column.split()).lower())
# Display our results.
stocks_df.show(5)

+----------+------------------+------------------+------------------+------------------+-------------------+--------+
|      date|              open|              high|               low|             close|          adj_close|  volume|
+----------+------------------+------------------+------------------+------------------+-------------------+--------+
|1962-01-02|6.5321550369262695| 6.556184768676758|6.5321550369262695|6.5321550369262695| 1.5366575717926023| 55900.0|
|1962-01-02| 6.125843524932861| 6.160982131958008| 6.125843524932861| 6.125843524932861| 1.4146506786346436| 59700.0|
|1962-01-02|0.8374485373497009|0.8374485373497009|0.8230452537536621|0.8230452537536621| 0.1457476019859314|352200.0|
|1962-01-02|1.6041666269302368|1.6197916269302368|1.5885416269302368|1.6041666269302368|0.13695742189884186|163200.0|
|1962-01-02|               0.0|3.2961308956146236|3.2440476417541504|3.2961308956146236| 0.0519925132393837|105600.0|
+----------+------------------+------------------+------

## Null Values
Null values often represent missing pieces of data. It is always good to know where your null values lie - so you can quickly identify and remedy any issues stemming from these.

> ℹ️ **Instructions** ℹ️
>
>Write code to count the number of null values found in each column.
>
>*You may use as many coding cells as necessary.*

In [21]:
#TODO: Write your code here
#from pyspark.sql.functions import col

In [ ]:
#predict session code
missing_count = {}  # Dictionary to keep track of the results
for column in new_df.columns:   # loop through each column
    _count = new_df.where(new_df[column].isNull()).count()  # null count in column x
    _total_count = new_df.select(new_df[column]).count()    # total count of column x 
    print(f'There are {_count} ({round(_count/_total_count*100, 3)}%) null values in {column} column')  # print out and calculate results
    missing_count[f'{column}'] = _count # recording results in missing_count dictionary 

In [24]:
# Count the number of null values in each column before conversion
null_counts_before = {col_name: stocks_df.where(col(col_name).isNull()).count() for col_name in stocks_df.columns}
null_counts_before

{'date': 0,
 'open': 0,
 'high': 0,
 'low': 0,
 'close': 0,
 'adj_close': 0,
 'volume': 0}

## Data type conversion - The final data schema

Now that we have identified the number of missing values in the data set, we'll move on to convert our data schema to the required data types. 

> ℹ️ **Instructions** ℹ️
>
>Use typecasting to convert the string data types in your current data schema to more appropriate data types.
>
>*You may use as many coding cells as necessary.*

In [ ]:
#TODO: Write your code here
#Predict session code
new_df = df.withColumn('date', F.col('date').cast(DateType()))
new_df = df.withColumn('open', F.col('open').cast(DoubleType()))
new_df = df.withColumn('high', F.col('high').cast(DateType()))
new_df = df.withColumn('low', F.col('low').cast(DateType()))
new_df = df.withColumn('close', F.col('close').cast(DateType()))
new_df = df.withColumn('adj_close', F.col('adj_close').cast(DateType()))
new_df = df.withColumn('volume', F.col('volume').cast(DateType()))


In [ ]:
new_df.printSchema()

In [62]:
#TODO: Write your code here - Remove
stocks_schema = StructType([StructField('date', IntegerType(), True),
                            StructField('open', FloatType(), True),
                            StructField('high', FloatType(), True),
                            StructField('low', IntegerType(), True),
                            StructField('close', FloatType(), True), 
                            StructField('adj_close', FloatType(), True),
                            StructField('volume', FloatType(), True)
])


In [63]:
typed_df = spark.read.csv('C:/Users/Naomi/Desktop/data/stocks/1962/01/02/stocks.csv', header=True, schema=stocks_schema)
typed_df.show(10)
typed_df.printSchema()

+----+----------+-----------+----+----------+------------+---------+
|date|      open|       high| low|     close|   adj_close|   volume|
+----+----------+-----------+----+----------+------------+---------+
|null|  6.532155|   6.556185|null|  6.532155|   1.5366576|  55900.0|
|null| 6.1258435|   6.160982|null| 6.1258435|   1.4146507|  59700.0|
|null|0.83744854| 0.83744854|null|0.82304525|   0.1457476| 352200.0|
|null| 1.6041666|  1.6197916|null| 1.6041666|  0.13695742| 163200.0|
|null|       0.0|   3.296131|null|  3.296131| 0.051992513| 105600.0|
|null|0.09290839|0.096026115|null|0.09290839| 0.035517246| 817400.0|
|null|       0.0|    30.6875|null|    30.375|   0.5069433|   1600.0|
|null|       0.0|   10.28125|null|    10.125|  0.22735077|  25600.0|
|null|       0.0|     7.6875|null| 7.5833335|   0.9789446|  49200.0|
|null| 0.7512019| 0.76372194|null|0.74869794|0.0017815924|2156500.0|
+----+----------+-----------+----+----------+------------+---------+
only showing top 10 rows

root
 |-

## Consolidate missing values
We have to check if the data type conversion above was done correctly.
If the casting was not successful, a null value gets inserted into the dataframe. You can thus check for successful conversion by determining if any null values are included in the resulting dataframe.


> ℹ️ **Instructions** ℹ️
>
>Write code to compare the number of invalid entries (nulls) pre-conversion and post-conversion.
>
>*You may use as many coding cells as necessary.*

In [64]:
#TODO: Write your code here
null_counts_after = {col_name: typed_df.where(col(col_name).isNull()).count() for col_name in typed_df.columns}
null_counts_after

{'date': 20,
 'open': 0,
 'high': 0,
 'low': 20,
 'close': 0,
 'adj_close': 0,
 'volume': 0}

In [66]:
# Compare the number of invalid entries (nulls) pre-conversion and post-conversion
# Compare counts before and after conversion
print("Null Value Counts Before Conversion:")
for col_name, count in null_counts_before.items():
    print(f"Column '{col_name}': {count} null values")

print("\nNull Value Counts After Conversion:")
for col_name, count in null_counts_after.items():
    print(f"Column '{col_name}': {count} null values")

print("\nComparison:")
for col_name in typed_df.columns:
    difference = null_counts_after[col_name] - null_counts_before[col_name]
    print(f"Column '{col_name}': {difference} null value difference")

Null Value Counts Before Conversion:
Column 'date': 0 null values
Column 'open': 0 null values
Column 'high': 0 null values
Column 'low': 0 null values
Column 'close': 0 null values
Column 'adj_close': 0 null values
Column 'volume': 0 null values

Null Value Counts After Conversion:
Column 'date': 20 null values
Column 'open': 0 null values
Column 'high': 0 null values
Column 'low': 20 null values
Column 'close': 0 null values
Column 'adj_close': 0 null values
Column 'volume': 0 null values

Comparison:
Column 'date': 20 null value difference
Column 'open': 0 null value difference
Column 'high': 0 null value difference
Column 'low': 20 null value difference
Column 'close': 0 null value difference
Column 'adj_close': 0 null value difference
Column 'volume': 0 null value difference


Here you should be able to see if any of your casts went wrong. 
Do not attempt to correct any missing values at this point. This will be dealt with in later sections of the predict.

## Generate parquet files
When writing in Spark, we typically use parquet format. This format allows parallel writing using Spark's optimisation while maintaining other useful things like metadata.

When writing, it is good to make sure that the data is sufficiently partitioned. 

Generally, data should be partitioned with one partition for every 200MB of data, but this also depends on the size of your cluster and executors. 


### Check the size of the dataframe before partitioning

In [69]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

In [72]:
#new_df
rdd = typed_df.rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
obj = rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)
size = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(obj)
size_MB = size/1000000
partitions = max(int(size_MB/200), 2)
print(f'The dataframe is {size_MB} MB')

The dataframe is 15.751832 MB


2

### Write parquet files to the local directory
> ℹ️ **Instructions** ℹ️
>
> Use the **coalesce** function and the number of **partitions** derived above to write parquet files to your local directory 
>
>*You may use as many coding cells as necessary.*

In [ ]:
#predict session
new_def.coalesce(partitions).write.parquet('content')

In [81]:
#TODO: Write your code here
# Coalesce the DataFrame to the desired number of partitions
#num_partitions = 4  # Replace with the desired number of partitions
df_coalesced = typed_df.coalesce(partitions)

# Write the DataFrame to Parquet files in the local directory
output_path = "C:/Users/Naomi/Desktop/data/transformed_data"
df_coalesced.write.parquet(output_path)


Py4JJavaError: An error occurred while calling o536.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.j

In [80]:
df_coalesced.write.format("parquet").save("C:/Users/Naomi/Desktop/data/transformed_data1")

Py4JJavaError: An error occurred while calling o531.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java